In [1]:
# Best approach: create and cache a series of more and more sparsified values, each sorted by y, when needed just cut out the y section and supply it

import pandas
import numpy as np

database_loc = "./database/database.feather"

database = pandas.read_feather(database_loc)
database.set_index("node_id", inplace=True)

database['y'] = database['y'] * 20  #* 10000000000
database['x'] = database['x'] * 45  #* 1000 * 2**10


In [2]:
database

,x,y,parent_id,num_tips,name,meta_Country,meta_Lineage
node_id,,,,,,,
0,6.750000,0.000000,64464,1,AUS/NT49/2020,Australia,B
1,6.750000,0.000333,64464,1,AUS/VIC102/2020,Australia,B
2,6.750000,0.000667,64464,1,AUS/VIC96/2020,Australia,B
3,6.750000,0.001000,64464,1,CHN/Beijing-01/2020,China,B
4,6.750000,0.001333,64464,1,CHN/YN-0306-466/2020,China,B
...,...,...,...,...,...,...,...
4766569,330.749996,1193.064346,4766559,1,England/PHEC-3V085V42/2021,England,AY.4.2.2
4766570,330.749996,1193.064651,4766559,1,England/PHEC-YYB1ESF/2021,England,AY.4.2.2
4766571,330.749996,1193.065033,4766559,1,England/PHEC-YYB1PWR/2021,England,AY.4.2.2


In [30]:
is_usa = database['meta_Country'] == 'USA'
indices = database.index[is_usa]

new_df_of_indices = pandas.DataFrame(indices)
new_df_of_indices.to_feather("/tmp/indices.feather")
%time new_df_of_indices = pandas.read_feather("/tmp/indices.feather")

CPU times: user 23.6 ms, sys: 45.8 ms, total: 69.4 ms
Wall time: 108 ms


In [86]:
import json
cache = {}
def matching_nodes(searchSpec):
    json_searchSpec = json.dumps(searchSpec)
    if json_searchSpec in cache:
        return cache[json_searchSpec]
    if searchSpec['type'] == "metadata":
        category = searchSpec['category']
        matches = database.loc[database[category] == searchSpec['value']]
        indices = matches.index
        cache[json_searchSpec] = indices

In [92]:
just = as_np[:,0:2]
# force to int32
just = just.astype(np.int32)

In [90]:
def get_unique(x):
    x2 = np.ascontiguousarray(x).view(np.dtype((np.void, x.dtype.itemsize * x.shape[1])))
    y_temp, z = np.unique(x2, return_inverse=True)
    y = y_temp.view(dtype='int64').reshape(len(y_temp), 2)

In [101]:
just2 = just.round(2)
%time uniqued = np.unique(just2, return_index=True,axis=0) 
uniqued[1]

CPU times: user 2.46 s, sys: 49.3 ms, total: 2.51 s
Wall time: 2.49 s


array([   3677,   64464,       0, ...,   21583, 1484745, 2302460])

In [109]:
import pandas
import numpy as np

database_loc = "./database/database.feather"

database = pandas.read_feather(database_loc)
database.set_index("node_id", inplace=True)

database['y'] = database['y'] * 20  #* 10000000000
database['x'] = database['x'] * 45  #* 1000 * 2**10

%time database['x'] = database.x.round(2)
%time database['y'] = database.y.round(2)

CPU times: user 15.7 ms, sys: 10.9 ms, total: 26.6 ms
Wall time: 26.6 ms
CPU times: user 27.3 ms, sys: 0 ns, total: 27.3 ms
Wall time: 27.1 ms


In [114]:
import pandas as pd
%time b= database.drop_duplicates(subset=['x','y'], keep='first', inplace=False)

CPU times: user 229 ms, sys: 0 ns, total: 229 ms
Wall time: 227 ms


In [116]:
%%time
for x in database.x:
    pass

CPU times: user 233 ms, sys: 8.77 ms, total: 241 ms
Wall time: 239 ms
